In [ ]:
import os
import numpy as np
import pandas as pd
from numpy import genfromtxt
from tensorflow.keras.preprocessing import image

from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.applications import ResNet50                                   ### Import resnet50 as our pre-trained model for CNN
from keras.layers import BatchNormalization

In [ ]:
trainLabelscsv = genfromtxt('/content/drive/MyDrive/Diabetic/trainDR2.csv',dtype=str, delimiter=',')

In [ ]:
trainLabelscsv.shape[0]

3663

In [ ]:
PATH='/content/drive/MyDrive/Diabetic/DR2/'
files=os.listdir(PATH)
img_data_list=[]
labels=[]

In [ ]:
def preprocessImageData():
    y=trainLabelscsv.shape[0]
    for i in files:
        try:
            img_path = PATH + i
            img = image.load_img(img_path, target_size=(224, 224))
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            # print('Input image shape:', x.shape)
            img_data_list.append(x)
            for j in range(0,y):
                if i==trainLabelscsv[j][0]+'.png':
                        # print(trainLabelscsv[j][1])
                        labels.append(trainLabelscsv[j][1])
        except:   
            pass   

In [ ]:
preprocessImageData()
X = np.array(img_data_list)
print(X.shape)

(3662, 1, 224, 224, 3)


In [ ]:
type(X)

numpy.ndarray

In [ ]:
X=np.rollaxis(X,1,0)
print(X)

[[[[[-101.939 -116.779 -121.68 ]
    [-101.939 -116.779 -121.68 ]
    [-101.939 -116.779 -121.68 ]
    ...
    [-101.939 -116.779 -121.68 ]
    [-102.939 -116.779 -122.68 ]
    [-102.939 -116.779 -122.68 ]]

   [[-101.939 -116.779 -121.68 ]
    [-101.939 -116.779 -121.68 ]
    [-101.939 -116.779 -121.68 ]
    ...
    [-102.939 -116.779 -122.68 ]
    [-102.939 -116.779 -122.68 ]
    [-101.939 -116.779 -121.68 ]]

   [[-101.939 -116.779 -121.68 ]
    [-101.939 -116.779 -121.68 ]
    [-101.939 -116.779 -121.68 ]
    ...
    [-102.939 -116.779 -122.68 ]
    [-102.939 -116.779 -122.68 ]
    [-101.939 -116.779 -121.68 ]]

   ...

   [[-101.939 -116.779 -121.68 ]
    [-101.939 -116.779 -121.68 ]
    [-101.939 -116.779 -121.68 ]
    ...
    [-101.939 -116.779 -121.68 ]
    [-102.939 -116.779 -122.68 ]
    [-101.939 -116.779 -121.68 ]]

   [[-101.939 -116.779 -121.68 ]
    [-101.939 -116.779 -121.68 ]
    [-101.939 -116.779 -121.68 ]
    ...
    [-101.939 -116.779 -121.68 ]
    [-102.939 -116.7

In [ ]:
X=X[0]
print (X.shape)

(3662, 224, 224, 3)


In [ ]:
num_of_samples = X.shape[0]

In [ ]:
num_of_samples

3662

In [ ]:
# labels

In [ ]:
num_classes=5
Y = np_utils.to_categorical(labels, num_classes)

In [ ]:
type(Y)

numpy.ndarray

In [ ]:
X = X.astype('float32')/255  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=40)

In [ ]:
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=40)

In [ ]:
model = Sequential()

model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.layers[0].trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 batch_normalization_1 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 1024)              2

In [ ]:
lr = 0.001
epochs = 100
bs = 20
optimizer = Adam(lr = lr, decay= lr/epochs)
model.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15,fill_mode ="nearest")

In [ ]:
checkpointer = ModelCheckpoint(filepath = "/content/drive/MyDrive/Diabetic/model.h5", save_best_only = True, verbose=1)

In [ ]:
history=model.fit(train_datagen.flow(X_train, y_train, batch_size = bs),
                            steps_per_epoch = len(X_train)//bs,
                            validation_data = (X_valid, y_valid),
                            validation_steps = len(X_valid)//bs,
                            epochs =epochs,
                            callbacks= [checkpointer])

Epoch 1/100
146/146 [==============================] - ETA: 0s - loss: 0.4408 - accuracy: 0.6525
Epoch 1: val_loss improved from inf to 0.40533, saving model to /content/drive/MyDrive/Diabetic/model.h5
146/146 [==============================] - 59s 331ms/step - loss: 0.4408 - accuracy: 0.6525 - val_loss: 0.4053 - val_accuracy: 0.5000
Epoch 2/100
146/146 [==============================] - ETA: 0s - loss: 0.2874 - accuracy: 0.6961
Epoch 2: val_loss improved from 0.40533 to 0.39096, saving model to /content/drive/MyDrive/Diabetic/model.h5
146/146 [==============================] - 41s 279ms/step - loss: 0.2874 - accuracy: 0.6961 - val_loss: 0.3910 - val_accuracy: 0.4727
Epoch 3/100
146/146 [==============================] - ETA: 0s - loss: 0.2779 - accuracy: 0.6937
Epoch 3: val_loss improved from 0.39096 to 0.34594, saving model to /content/drive/MyDrive/Diabetic/model.h5
146/146 [==============================] - 41s 278ms/step - loss: 0.2779 - accuracy: 0.6937 - val_loss: 0.3459 - val_a

In [ ]:
(eval_loss, eval_accuracy) = model.evaluate(  
     X_test, y_test, batch_size=bs, verbose=2)

19/19 - 2s - loss: 0.3405 - accuracy: 0.7357 - 2s/epoch - 86ms/step


In [ ]:
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {:.2f}%".format(eval_loss*100))

Accuracy: 73.57%
Loss: 34.05%
